**VGG NN**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import models

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# CIFAR-10 dataset has 10 classes
num_classes = 10

In [ ]:
# Define the VGG block
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [ ]:
# Define the VGG network
class VGG(nn.Module):
    def __init__(self, conv_arch, num_classes=10):
        super(VGG, self).__init__()
        self.conv_layers = self._make_layers(conv_arch)
        self.fc_layers = nn.Sequential(
            nn.Linear(conv_arch[-1][1] * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
    def _make_layers(self, conv_arch):
        layers = []
        in_channels = 3
        for (num_convs, out_channels) in conv_arch:
            layers.append(vgg_block(num_convs, in_channels, out_channels))
            in_channels = out_channels
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the features
        x = self.fc_layers(x)
        return x

In [ ]:
# Define the VGG architecture
conv_arch = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512))

In [ ]:
model = VGG(conv_arch, num_classes=num_classes).to(device)


In [ ]:
# Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 10


# Image preprocessing modules
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
# CIFAR-10 data
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)


100%|██████████| 170498071/170498071 [00:01<00:00, 85908975.07it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Function for training the model
def train(model, criterion, optimizer, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


# Function for testing the model
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Accuracy of the model on the test images: {accuracy}%')


# Train the model
train(model, criterion, optimizer, train_loader, epochs)


# Test the model
test(model, test_loader)


Epoch [1/10], Step [100/782], Loss: 2.3235
Epoch [1/10], Step [200/782], Loss: 2.0200
Epoch [1/10], Step [300/782], Loss: 1.8915
Epoch [1/10], Step [400/782], Loss: 1.8704
Epoch [1/10], Step [500/782], Loss: 1.9583
Epoch [1/10], Step [600/782], Loss: 1.8693
Epoch [1/10], Step [700/782], Loss: 1.8328
Epoch [2/10], Step [100/782], Loss: 1.7250
Epoch [2/10], Step [200/782], Loss: 2.0903
Epoch [2/10], Step [300/782], Loss: 1.9484
Epoch [2/10], Step [400/782], Loss: 1.5514
Epoch [2/10], Step [500/782], Loss: 1.5390
Epoch [2/10], Step [600/782], Loss: 1.6595
Epoch [2/10], Step [700/782], Loss: 1.3919
Epoch [3/10], Step [100/782], Loss: 1.8727
Epoch [3/10], Step [200/782], Loss: 1.5932
Epoch [3/10], Step [300/782], Loss: 1.5194
Epoch [3/10], Step [400/782], Loss: 1.4973
Epoch [3/10], Step [500/782], Loss: 1.3845
Epoch [3/10], Step [600/782], Loss: 1.4995
Epoch [3/10], Step [700/782], Loss: 1.3093
Epoch [4/10], Step [100/782], Loss: 1.2698
Epoch [4/10], Step [200/782], Loss: 1.3011
Epoch [4/10

**Googlenet**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [ ]:
device=torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [ ]:
# CIFAR-10 has 10 classes
num_classes = 10


# Image Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])


In [ ]:
# Download and load the CIFAR-10 data
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)


# Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


# Define a GoogLeNet model for CIFAR-10
class GoogLeNetCIFAR10(models.GoogLeNet):
    def __init__(self, num_classes=10):
        super(GoogLeNetCIFAR10, self).__init__(num_classes=num_classes, aux_logits=False, init_weights=True)
        # Redefine the first convolutional layer to accept CIFAR-10's 32x32 pixel images
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Follow the forward pass of the GoogLeNet model
        return super(GoogLeNetCIFAR10, self).forward(x)




Files already downloaded and verified


In [ ]:
# Instantiate the model and transfer it to the GPU
model = GoogLeNetCIFAR10(num_classes=num_classes).to(device)


# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Function to train the model
def train(model, criterion, optimizer, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            # Transfer inputs and labels to the GPU
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
                running_loss = 0.0


# Function to test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            # Transfer images and labels to the GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


# Start training
train(model, criterion, optimizer, train_loader, epochs=10)


# Test the model
accuracy = test(model, test_loader)
print(f'Accuracy of the model on the 10000 test images: {accuracy}%')


Epoch 1, Batch 100, Loss: 1.930627955198288
Epoch 1, Batch 200, Loss: 1.676651691198349
Epoch 1, Batch 300, Loss: 1.5533457851409913
Epoch 1, Batch 400, Loss: 1.4630921161174775
Epoch 1, Batch 500, Loss: 1.3711948657035828
Epoch 1, Batch 600, Loss: 1.2865696090459824
Epoch 1, Batch 700, Loss: 1.231417679786682
Epoch 2, Batch 100, Loss: 1.101140775680542
Epoch 2, Batch 200, Loss: 1.0728317183256149
Epoch 2, Batch 300, Loss: 1.0463483840227128
Epoch 2, Batch 400, Loss: 0.9674238514900207
Epoch 2, Batch 500, Loss: 0.9371637552976608
Epoch 2, Batch 600, Loss: 0.9267370021343231
Epoch 2, Batch 700, Loss: 0.9096721619367599
Epoch 3, Batch 100, Loss: 0.814417108297348
Epoch 3, Batch 200, Loss: 0.7593256893754006
Epoch 3, Batch 300, Loss: 0.7928840535879135
Epoch 3, Batch 400, Loss: 0.7547766011953354
Epoch 3, Batch 500, Loss: 0.7206918570399284
Epoch 3, Batch 600, Loss: 0.7270461264252662
Epoch 3, Batch 700, Loss: 0.7006382346153259
Epoch 4, Batch 100, Loss: 0.6084755820035934
Epoch 4, Batch 